In [3]:
import cv2
import numpy as np

# Read and display video from webcam
frameWidth = 640
frameHeight = 480
cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)
cap.set(10, 150)

# HSV ranges for pink, blue, green
myColors = [
    [140, 100, 100, 170, 255, 255],  # Pink
    [100, 150, 100, 130, 255, 255],  # Blue
    [45, 150, 150, 75, 255, 255]     # Green
]

# Corresponding BGR color values to draw
myColorValues = [
    [255, 0, 255],  # Pink
    [255, 0, 0],    # Blue
    [0, 255, 0]     # Green
]

myPoints = [[] for _ in range(len(myColors))]  # One list for each color

def findColor(img, myColors, myColorValues):
    imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    count = 0
    newPoints = []
    for color in myColors:
        lower = np.array(color[0:3])
        upper = np.array(color[3:6])
        mask = cv2.inRange(imgHSV, lower, upper)
        x, y = getContours(mask)
        if x != 0 and y != 0:
            newPoints.append([x, y, count])
        count += 1
    return newPoints

def getContours(img):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, w, h = 0, 0, 0, 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 500:
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
            x, y, w, h = cv2.boundingRect(approx)
    return x + w // 2, y

def drawOnCanvas(myPoints, myColorValues):
    for i in range(1, len(myPoints)):
        if myPoints[i][2] == myPoints[i - 1][2]:  # Same color
            cv2.line(imgResult,
                     (myPoints[i - 1][0], myPoints[i - 1][1]),
                     (myPoints[i][0], myPoints[i][1]),
                     myColorValues[myPoints[i][2]], thickness=5)

if not cap.isOpened():
    print("Error: Could not open video!")
else:
    while True:
        success, img = cap.read()
        imgResult = img.copy()
        newPoints = findColor(img, myColors, myColorValues)
        if len(newPoints) != 0:
            for newP in newPoints:
                x, y, colorID = newP
                myPoints[colorID].append((x, y))
        for colorID in range(len(myPoints)):
            for i in range(1, len(myPoints[colorID])):
                cv2.line(imgResult, myPoints[colorID][i - 1], myPoints[colorID][i],
                         myColorValues[colorID], thickness=5)

        if not success:
            break
        cv2.imshow("Result", imgResult)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        elif cv2.waitKey(1) & 0xFF == ord('c'):
            myPoints = [[] for _ in range(len(myColors))]

    cap.release()
    cv2.destroyAllWindows()
